# PyTorch vs Keras VGG Comparison

## Imports

In [1]:
# ----- PYTORCH -----
import torch
from torch.autograd import Variable
from torchvision import models
import sys
import numpy as np
import torchvision
import torch.nn as nn
import torch.optim as optim
import argparse
import time
import tensorly as tl
from tensorly.decomposition import partial_tucker
from decompositions import cp_decomposition_conv_layer, tucker_decomposition_conv_layer

from VBMF import VBMF

from torch.utils.data import Dataset, DataLoader, random_split

from torch.nn.utils.rnn import pad_sequence

import torch.backends.cudnn as cudnn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data as data
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn.utils.rnn as rnn_utils
import torch.nn.functional as F
from torchvision.io import read_image
from PIL import Image
import glob
import os

import matplotlib.pyplot as plt
plt.ion()   # interactive mode

import pandas as pd
import numpy as np
from glob import glob
import os, os.path
import matplotlib.pyplot as plt
from numpy import asarray

import sklearn.metrics
from sklearn.metrics import accuracy_score, mean_squared_error, confusion_matrix, roc_auc_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC, SVR

from sksurv.datasets import load_gbsg2
# from sksurv.preprocessing import OneHotEncoder
from sksurv.ensemble import RandomSurvivalForest
from sksurv.metrics import concordance_index_censored, concordance_index_ipcw, integrated_brier_score

import itertools
from itertools import *

import datetime
import pickle

# # ----- KERAS -----
# import keras
# from keras.applications.vgg16 import VGG16
# from keras.applications.vgg16 import preprocess_input
# import numpy as np

# import time

# import os

# import PIL.Image as Image
# import matplotlib.pylab as plt

# import tensorflow as tf
# import tensorflow_hub as hub
# import tensorflow.keras.layers as tfl

# # import misc

# import pandas as pd

# import datetime

# import glob

# from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
# from tensorflow.keras import utils

# import keras
# from sklearn.model_selection import train_test_split
# # from tensorflow.keras.utils import ImageDataGenerator, img_to_array, load_img
# from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input
# from tensorflow.keras.models import Model, Sequential
# from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, GlobalAveragePooling2D
# from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
# # from keras.utils import np_utils
# from tensorflow.keras.optimizers import RMSprop
# from tensorflow.keras.layers import Concatenate
# from sklearn.datasets import load_files
# from sklearn.preprocessing import LabelBinarizer
# from sklearn.metrics import classification_report, confusion_matrix
# # import tensorflow_datasets as tfds
# from tensorflow.keras.callbacks import EarlyStopping

# from tensorflow.keras.preprocessing import image_dataset_from_directory

## PyTorch Device Selection

In [2]:
# Use GPU if possible
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

# Use PyTorch as Tensorly Backend
tl.set_backend('pytorch')

## PyTorch Model

### Declare Dataloader and Dataset

In [3]:
# Path to your dataset
dataset_path = "/home/mason/ADNI_Dataset/ADNI_IMG_32.5%_x_organized"

# Data transforms (resize, convert to tensor, normalize)
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),         # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

In [4]:
# Load dataset
dataset = datasets.ImageFolder(root=dataset_path, transform=data_transforms)

In [5]:
# Split into training and validation sets
train_size = int(0.8 * len(dataset))  # 80% for training
val_size = len(dataset) - train_size  # 20% for validation
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

In [6]:
# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

### Declare Model

In [7]:
class ModifiedVGG16Model(torch.nn.Module):
    def __init__(self, num_classes=3):
        super(ModifiedVGG16Model, self).__init__()

        model = models.vgg16(weights='IMAGENET1K_V1')
        self.features = model.features
        
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)  # GlobalAveragePooling2D
        
        self.shared = nn.Sequential( # Try simplififying these layers!
            nn.Flatten(),
            nn.Linear(512, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 256))
        
        # Contains the Tail of VGG16 (all 3 FC layers and ReLU, when combined with embedder)
        self.classifier = nn.Sequential(
            nn.Linear(256, num_classes))
        
    def forward(self, x):
        x = self.features(x)
        x = self.global_avg_pool(x)
        x = self.shared(x)
        x = self.classifier(x)

        return x

### Model Training

In [8]:
model = ModifiedVGG16Model()
model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [9]:
# Training loop
epochs = 30
for epoch in range(epochs):
    model.train()  # Set model to training mode
    train_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Track loss and accuracy
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    # Print training stats
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {train_loss/len(train_loader):.4f}, Accuracy: {correct/total:.4f}")

    # Validation loop
    model.eval()  # Set model to evaluation mode
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()

    print(f"Validation Loss: {val_loss/len(val_loader):.4f}, Validation Accuracy: {val_correct/val_total:.4f}")

Epoch 1/30, Loss: 1.2283, Accuracy: 0.4172
Validation Loss: 1.0660, Validation Accuracy: 0.4577
Epoch 2/30, Loss: 1.0620, Accuracy: 0.4476
Validation Loss: 1.0475, Validation Accuracy: 0.4577
Epoch 3/30, Loss: 1.0567, Accuracy: 0.4476
Validation Loss: 1.0473, Validation Accuracy: 0.4577
Epoch 4/30, Loss: 1.0558, Accuracy: 0.4476
Validation Loss: 1.0492, Validation Accuracy: 0.4577
Epoch 5/30, Loss: 1.0568, Accuracy: 0.4476
Validation Loss: 1.0483, Validation Accuracy: 0.4577
Epoch 6/30, Loss: 1.0576, Accuracy: 0.4424
Validation Loss: 1.0473, Validation Accuracy: 0.4577
Epoch 7/30, Loss: 1.0556, Accuracy: 0.4476
Validation Loss: 1.0472, Validation Accuracy: 0.4577
Epoch 8/30, Loss: 1.0554, Accuracy: 0.4476
Validation Loss: 1.0471, Validation Accuracy: 0.4577
Epoch 9/30, Loss: 1.0562, Accuracy: 0.4476
Validation Loss: 1.0456, Validation Accuracy: 0.4577
Epoch 10/30, Loss: 1.0547, Accuracy: 0.4476
Validation Loss: 1.0465, Validation Accuracy: 0.4577
Epoch 11/30, Loss: 1.0572, Accuracy: 0.

## Keras Model